# 📘 Agentic 架构 8：情境记忆 + 语义记忆栈

欢迎来到我们系列的第八个笔记本。今天，我们将应对构建真正智能、长期助理的最关键挑战之一：**持久记忆**。标准的聊天机器人记忆是短暂的，仅持续单个会话。要构建一个能够与用户共同学习和成长的个性化代理，我们需要更健壮的解决方案。

我们将实现一种模拟人类认知的结构化记忆架构，结合两种不同的记忆类型：

1.  **情境记忆（Episodic Memory）：** 这是对特定事件或过去交互的记忆。它回答的问题是：“发生了什么？”（例如，“上周，用户询问了我关于 NVIDIA 股价的问题。”）。我们将为此使用**向量数据库**，以查找与当前主题相关的过去对话。
2.  **语义记忆（Semantic Memory）：** 这是从那些事件中提取的结构化事实、概念和关系的记忆。它回答的问题是：“我知道什么？”（例如，“用户 Alex 是一位保守型投资者。”，“Alex 对科技股感兴趣。”）。我们将为此使用**图数据库（Neo4j）**，因为它擅长管理和查询复杂关系。

通过将两者结合，我们的代理不仅能够回忆过去的对话，还能为用户和世界构建一个丰富且相互关联的知识库，从而实现深度个性化和上下文感知的交互。

### 定义
**情境记忆 + 语义记忆栈**（Episodic + Semantic Memory Stack）是一种代理架构，它维护两种类型的长期记忆。**情境记忆**（Episodic Memory）存储经历的时序日志（例如，聊天历史摘要），通常基于语义相似性进行搜索。**语义记忆**（Semantic Memory）将提取出的结构化知识（事实、实体、关系）存储在知识库中，通常采用图数据库的形式。

### 高层工作流程

1.  **交互：** 代理与用户进行对话。
2.  **记忆检索（回忆）：** 对于新的用户查询，代理首先查询两种记忆系统。
    *   它在**情境**向量存储中搜索相似过去的对话。
    *   它在**语义**图数据库中查询与查询相关的实体和事实。
3.  **增强生成：** 检索到的记忆被添加到提示的上下文中，使 LLM 能够生成感知过去交互和已学习事实的响应。
4.  **记忆创建（编码）：** 交互完成后，后台进程分析对话。
    *   它创建本次对话的简洁摘要（新的**情境**记忆）。
    *   它提取关键实体和关系（新的**语义**记忆）。
5.  **记忆存储：** 新的情境摘要被嵌入并保存到向量存储中。新的语义事实以节点和边的形式写入图数据库。

### 何时使用 / 应用场景
*   **长期个人助理：** 能够记住你的偏好、项目和个人细节，持续数周或数月的助理。
*   **个性化系统：** 记住你的风格的电商机器人，或记住你的学习进度和薄弱环节的教育导师。
*   **复杂研究代理：** 在探索文档时构建主题知识图谱的代理，从而能够回答复杂的多跳问题。

### 优势与劣势
*   **优势：**
    *   **真正的个性化：** 实现上下文和学习能够无限期持久，远远超出单个会话的上下文窗口。
    *   **丰富的理解能力：** 图数据库使代理能够理解并推理实体之间的复杂关系。
*   **劣势：**
    *   **复杂性：** 相比简单的无状态代理，这种架构的构建和维护复杂度显著更高。
    *   **记忆膨胀与修剪：** 随着时间推移，记忆存储可能会变得庞大。需要采用摘要、整合或修剪旧的/无关记忆的策略，以确保长期性能。

## 第零阶段：基础与设置

我们将安装所有必要的库，包括向量数据库和图数据库的驱动程序，并配置我们的 API 密钥。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain_community langchain-openai neo4j faiss-cpu tiktoken

In [2]:
import os
import uuid
from typing import List, Dict, Any, Optional, Tuple
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius, NebiusEmbeddings
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Memory Stack (Nebius)"

# Check for required environment variables
required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## 第一阶段：构建记忆组件

这是我们架构的核心部分。我们将定义记忆的结构，设置与数据库的连接。同时，我们还将创建负责处理对话并生成新记忆的“Memory Maker”（记忆创建器）代理。

In [3]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
embeddings = NebiusEmbeddings()

# --- 1. Vector Store for Episodic Memory ---
# In a real application, you'd persist this. For this example, it's in-memory.
try:
    episodic_vector_store = FAISS.from_texts(["Initial document to bootstrap the store"], embeddings)
except ImportError:
    console.print("[bold red]FAISS not installed. Please run `pip install faiss-cpu`.")[/bold red]
    episodic_vector_store = None

# --- 2. Graph DB for Semantic Memory ---
try:
    graph = Neo4jGraph(
        url=os.environ.get("NEO4J_URI"),
        username=os.environ.get("NEO4J_USERNAME"),
        password=os.environ.get("NEO4J_PASSWORD")
    )
    # Clear the graph for a clean run
    graph.query("MATCH (n) DETACH DELETE n")
except Exception as e:
    console.print(f"[bold red]Failed to connect to Neo4j: {e}. Please check your credentials and connection.[/bold red]")
    graph = None

# --- 3. Pydantic Models for the "Memory Maker" ---
# Define the structure of knowledge we want to extract.
class Node(BaseModel):
    id: str = Field(description="Unique identifier for the node, which can be a person's name, a company ticker, or a concept.")
    type: str = Field(description="The type of the node (e.g., 'User', 'Company', 'InvestmentPhilosophy').")
    properties: Dict[str, Any] = Field(description="A dictionary of properties for the node.")

class Relationship(BaseModel):
    source: Node = Field(description="The source node of the relationship.")
    target: Node = Field(description="The target node of the relationship.")
    type: str = Field(description="The type of the relationship (e.g., 'IS_A', 'INTERESTED_IN').")
    properties: Dict[str, Any] = Field(description="A dictionary of properties for the relationship.")

class KnowledgeGraph(BaseModel):
    """Represents the structured knowledge extracted from a conversation."""
    relationships: List[Relationship] = Field(description="A list of relationships to be added to the knowledge graph.")

# --- 4. The "Memory Maker" Agent ---
def create_memories(user_input: str, assistant_output: str):
    conversation = f"User: {user_input}\nAssistant: {assistant_output}"
    
    # 4a. Create Episodic Memory (Summarization)
    console.print("--- Creating Episodic Memory (Summary) ---")
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a summarization expert. Create a concise, one-sentence summary of the following user-assistant interaction. This summary will be used as a memory for future recall."),
        ("human", "Interaction:\n{interaction}")
    ])
    summarizer = summary_prompt | llm
    episodic_summary = summarizer.invoke({"interaction": conversation}).content
    
    new_doc = Document(page_content=episodic_summary, metadata={"created_at": uuid.uuid4().hex})
    episodic_vector_store.add_documents([new_doc])
    console.print(f"[green]Episodic memory created:[/green] '{episodic_summary}'")
    
    # 4b. Create Semantic Memory (Fact Extraction)
    console.print("--- Creating Semantic Memory (Graph) ---")
    extraction_llm = llm.with_structured_output(KnowledgeGraph)
    extraction_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a knowledge extraction expert. Your task is to identify key entities and their relationships from a conversation and model them as a graph. Focus on user preferences, goals, and stated facts."),
        ("human", "Extract all relationships from this interaction:\n{interaction}")
    ])
    extractor = extraction_prompt | extraction_llm
    try:
        kg_data = extractor.invoke({"interaction": conversation})
        if kg_data.relationships:
            for rel in kg_data.relationships:
                graph.add_graph_documents([rel], include_source=True)
            console.print(f"[green]Semantic memory created:[/green] Added {len(kg_data.relationships)} relationships to the graph.")
        else:
            console.print("[yellow]No new semantic memories identified in this interaction.[/yellow]")
    except Exception as e:
        console.print(f"[red]Could not extract or save semantic memory: {e}[/red]")

if episodic_vector_store and graph:
    print("Memory components initialized successfully.")

Memory components initialized successfully.


## 第二阶段：记忆增强代理

现在，我们将构建使用此记忆系统的代理。我们将使用 LangGraph 来定义一个清晰的有状态工作流程：检索记忆、使用这些记忆生成响应，最后用最新的交互更新记忆。

In [4]:
# Define the state for our LangGraph agent
class AgentState(TypedDict):
    user_input: str
    retrieved_memories: Optional[str]
    generation: str

# Define the nodes of the graph

def retrieve_memory(state: AgentState) -> Dict[str, Any]:
    """Node that retrieves memories from both episodic and semantic stores."""
    console.print("--- Retrieving Memories ---")
    user_input = state['user_input']
    
    # Retrieve from episodic memory
    retrieved_docs = episodic_vector_store.similarity_search(user_input, k=2)
    episodic_memories = "\n".join([doc.page_content for doc in retrieved_docs])
    
    # Retrieve from semantic memory
    # This is a simple retrieval; more advanced would involve entity extraction from the query
    try:
        graph_schema = graph.get_schema
        # Using a fulltext index for better retrieval. Neo4j automatically creates one on node properties.
        # A more robust solution might involve extracting entities from user_input first.
        semantic_memories = str(graph.query("""
            UNWIND $keywords AS keyword
            CALL db.index.fulltext.queryNodes("entity", keyword) YIELD node, score
            MATCH (node)-[r]-(related_node)
            RETURN node, r, related_node LIMIT 5
            """, {'keywords': user_input.split()}))
    except Exception as e:
        semantic_memories = f"Could not query graph: {e}"
        
    retrieved_content = f"Relevant Past Conversations (Episodic Memory):\n{episodic_memories}\n\nRelevant Facts (Semantic Memory):\n{semantic_memories}"
    console.print(f"[cyan]Retrieved Context:\n{retrieved_content}[/cyan]")
    
    return {"retrieved_memories": retrieved_content}

def generate_response(state: AgentState) -> Dict[str, Any]:
    """Node that generates a response using the retrieved memories."""
    console.print("--- Generating Response ---")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful and personalized financial assistant. Use the retrieved memories to inform your response and tailor it to the user. If the memories indicate a user's preference (e.g., they are a conservative investor), you MUST respect it."),
        ("human", "My question is: {user_input}\n\nHere are some memories that might be relevant:\n{retrieved_memories}")
    ])
    generator = prompt | llm
    generation = generator.invoke(state).content
    console.print(f"[green]Generated Response:\n{generation}[/green]")
    return {"generation": generation}

def update_memory(state: AgentState) -> Dict[str, Any]:
    """Node that updates the memory with the latest interaction."""
    console.print("--- Updating Memory ---")
    create_memories(state['user_input'], state['generation'])
    return {}

# Build the graph
workflow = StateGraph(AgentState)

workflow.add_node("retrieve", retrieve_memory)
workflow.add_node("generate", generate_response)
workflow.add_node("update", update_memory)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", "update")
workflow.add_edge("update", END)

memory_agent = workflow.compile()
print("Memory-augmented agent graph compiled successfully.")

Memory-augmented agent graph compiled successfully.


## 第三阶段：演示与检查

让我们来看看代理的实际运行表现。我们将模拟一个多轮对话。前两轮对话将用于初始化记忆。第三轮对话将测试代理是否能够利用这些记忆给出个性化的响应。最后，我们将直接检查数据库，查看所创建的记忆内容。

In [5]:
def run_interaction(query: str):
    result = memory_agent.invoke({"user_input": query})
    return result['generation']

console.print("\n--- 💬 INTERACTION 1: Seeding Memory ---")
run_interaction("Hi, my name is Alex. I'm a conservative investor, and I'm mainly interested in established tech companies.")

console.print("\n--- 💬 INTERACTION 2: Asking a specific question ---")
run_interaction("What do you think about Apple (AAPL)?")

console.print("\n--- 🧠 INTERACTION 3: THE MEMORY TEST ---")
run_interaction("Based on my goals, what's a good alternative to that stock?")


--- 💬 INTERACTION 1: Seeding Memory ---


--- Retrieving Memories ---
Retrieved Context:
Relevant Past Conversations (Episodic Memory):
Initial document to bootstrap the store

Relevant Facts (Semantic Memory):
[]
--- Generating Response ---
Generated Response:
Hello, Alex! It's great to meet you. As a conservative investor, focusing on established tech companies with strong fundamentals is a very sound strategy. I can certainly help you navigate that space. What's on your mind today?
--- Updating Memory ---
--- Creating Episodic Memory (Summary) ---
Episodic memory created: 'The user, Alex, introduced himself as a conservative investor interested in established tech companies.'
--- Creating Semantic Memory (Graph) ---
Semantic memory created: Added 2 relationships to the graph.



--- 💬 INTERACTION 2: Asking a specific question ---


--- Retrieving Memories ---
Retrieved Context:
Relevant Past Conversations (Episodic Memory):
The user, Alex, introduced himself as a conservative investor interested in established tech companies.
Initial document to bootstrap the store

Relevant Facts (Semantic Memory):
[]
--- Generating Response ---
Generated Response:
Apple (AAPL) is often considered a cornerstone for conservative tech portfolios. It has a massive market capitalization, a very strong brand, consistent profitability, and a history of returning value to shareholders through dividends and buybacks. Its ecosystem of products creates a loyal customer base, which provides a stable revenue stream. For a conservative investor, it generally aligns well with the goal of capital preservation while still offering growth potential. Would you like a deeper dive into its recent performance or financials?
--- Updating Memory ---
--- Creating Episodic Memory (Summary) ---
Episodic memory created: 'The user inquired about Apple (AAP


--- 🧠 INTERACTION 3: THE MEMORY TEST ---


--- Retrieving Memories ---
Retrieved Context:
Relevant Past Conversations (Episodic Memory):
The user, Alex, introduced himself as a conservative investor interested in established tech companies.
The user inquired about Apple (AAPL), and the assistant confirmed it's a suitable stock for conservative investors due to its stability and market position.

Relevant Facts (Semantic Memory):
[{'node': {'type': 'User', 'id': 'Alex', 'properties': {}}, 'r': {'type': 'HAS_GOAL', 'properties': {}}, 'related_node': {'type': 'InvestmentPhilosophy', 'id': 'Conservative Investing', 'properties': {}}}, {'node': {'type': 'User', 'id': 'Alex', 'properties': {}}, 'r': {'type': 'INTERESTED_IN', 'properties': {}}, 'related_node': {'type': 'Sector', 'id': 'Tech', 'properties': {}}}]
--- Generating Response ---
Generated Response:
Of course. Based on your stated goal of conservative investing in the tech sector, a great alternative to Apple (AAPL) would be Microsoft (MSFT).

Here's why it fits your profile

### 检查记忆存储

让我们来揭开盖子。我们可以直接查询数据库，查看代理创建的记忆内容。

In [6]:
console.print("--- 🔍 Inspecting Episodic Memory (Vector Store) ---")
# We'll do a similarity search for a general concept to see what comes up
retrieved_docs = episodic_vector_store.similarity_search("User's investment strategy", k=3)
for i, doc in enumerate(retrieved_docs):
    print(f"{i+1}. {doc.page_content}")

console.print("\n--- 🕸️ Inspecting Semantic Memory (Graph Database) ---")
print(f"Graph Schema:\n{graph.get_schema}")

# Cypher query to see who is interested in what
query_result = graph.query("MATCH (n:User)-[r:INTERESTED_IN|HAS_GOAL]->(m) RETURN n, r, m")
print(f"Relationships in Graph:\n{query_result}")

--- 🔍 Inspecting Episodic Memory (Vector Store) ---


1. The user, Alex, introduced himself as a conservative investor interested in established tech companies.
2. Based on the user's conservative investment goals, the assistant suggested Microsoft (MSFT) as a good alternative to Apple (AAPL), highlighting its diversification and enterprise focus.
3. The user inquired about Apple (AAPL), and the assistant confirmed it's a suitable stock for conservative investors due to its stability and market position.



--- 🕸️ Inspecting Semantic Memory (Graph Database) ---


Graph Schema:
{'node_props': {'InvestmentPhilosophy': [{'property': 'id', 'type': 'STRING'}], 'Company': [{'property': 'id', 'type': 'STRING'}], 'Sector': [{'property': 'id', 'type': 'STRING'}], 'User': [{'property': 'id', 'type': 'STRING'}]}, 'rel_props': {}, 'relationships': [{'start': 'User', 'type': 'INTERESTED_IN', 'end': 'Sector'}, {'start': 'User', 'type': 'INTERESTED_IN', 'end': 'Company'}, {'start': 'User', 'type': 'HAS_GOAL', 'end': 'InvestmentPhilosophy'}]}

Relationships in Graph:
[{'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'Conservative Investing', 'type': 'InvestmentPhilosophy'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'Tech', 'type': 'Sector'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'AAPL', 'type': 'Company'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'MSFT', 'type': 'Company'}}]


## 结论

在本笔记本中，我们成功构建了一个具备复杂长期记忆系统的代理。演示清楚地展示了这种架构的强大之处：

- **无状态失败：** 一个标准代理在被问到“基于我的目标，有什么好的替代方案？”时会失败，因为它对用户目标没有任何记忆。
- **记忆增强成功：** 我们的代理之所以成功，是因为它能够：
    1.  **情境回忆：** 检索到第一次对话的摘要：“用户 Alex 自我介绍为一位保守型投资者……”
    2.  **语义回忆：** 查询图数据库并找到结构化事实：`(User: Alex) -[HAS_GOAL]-> (InvestmentPhilosophy: Conservative)`。
    3.  **合成：** 利用这些组合上下文，提供高度相关且个性化的推荐（Microsoft），并明确提及用户的保守型投资目标。

这种结合回忆*发生了什么*（情境记忆）和*知道什么*（语义记忆）的模式，是一个强大的范式，它使我们能够超越简单的、事务性的代理，创造出真正的、会学习的伴侣。虽然在大规模管理这种记忆时会面临修剪和整合等挑战，但我们在这里构建的基础架构，是迈向更智能、更个性化 AI 系统的重要一步。